# import dependencies

In [2]:
import cv2
import numpy as np
import os
import matplotlib as plt
import time
import mediapipe as mp

# Keypoint values using mediapoint holistic

In [3]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [4]:
def mediapipe_detection (image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #conversion of collected image of bgr to rgb
    image.flags.writeable = False                  #image is not writeab
    results = model.process(image)                 #make predictions     
    image.flags.writeable = True                    #image is writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #conversion back to bgr
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [6]:
mp_drawing.draw_landmarks??

In [7]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness = 1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [8]:
cap = cv2.VideoCapture(0)  #accessing web cam

#setting mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened(): 
        
        #read from camera
        ret, frame = cap.read()

        #make detections using mediapipe_detection
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)
        
        #show to screen
        cv2.imshow('OpenCV Feed', image)

        #if we hit q then our video capture stops
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break        
    cap.release() #release webcam
    cv2.destroyAllWindows() #destroy the window

# Extracting Keypoint Values

In [9]:
len(results.face_landmarks.landmark)

468

In [10]:
results.face_landmarks

landmark {
  x: 0.5556417107582092
  y: 0.5506511926651001
  z: -0.041709065437316895
}
landmark {
  x: 0.5564379692077637
  y: 0.49486371874809265
  z: -0.061271052807569504
}
landmark {
  x: 0.5550739765167236
  y: 0.5160269737243652
  z: -0.03545897826552391
}
landmark {
  x: 0.5443669557571411
  y: 0.4431714117527008
  z: -0.04127802699804306
}
landmark {
  x: 0.556384801864624
  y: 0.47706303000450134
  z: -0.06365281343460083
}
landmark {
  x: 0.555531919002533
  y: 0.45605728030204773
  z: -0.05701319873332977
}
landmark {
  x: 0.5528959631919861
  y: 0.4083394408226013
  z: -0.019774919375777245
}
landmark {
  x: 0.4629240036010742
  y: 0.4167475700378418
  z: 0.01962009072303772
}
landmark {
  x: 0.5520967245101929
  y: 0.3647972345352173
  z: -0.008049188181757927
}
landmark {
  x: 0.5521266460418701
  y: 0.3401969075202942
  z: -0.007823665626347065
}
landmark {
  x: 0.5502923727035522
  y: 0.26206833124160767
  z: 0.013747435063123703
}
landmark {
  x: 0.5556065440177917
  

In [11]:
results.left_hand_landmarks

landmark {
  x: 0.9168175458908081
  y: 0.8199411034584045
  z: -8.603358310210751e-07
}
landmark {
  x: 0.8978914618492126
  y: 0.7310380935668945
  z: 0.029077069833874702
}
landmark {
  x: 0.9048449993133545
  y: 0.6494345664978027
  z: 0.03217550367116928
}
landmark {
  x: 0.9067673087120056
  y: 0.5872477889060974
  z: 0.024735528975725174
}
landmark {
  x: 0.8962949514389038
  y: 0.5374519228935242
  z: 0.015674371272325516
}
landmark {
  x: 0.9644688367843628
  y: 0.5623746514320374
  z: 0.02096366137266159
}
landmark {
  x: 0.8983181118965149
  y: 0.4643697440624237
  z: -0.0017419375944882631
}
landmark {
  x: 0.8378749489784241
  y: 0.47153419256210327
  z: -0.017270298674702644
}
landmark {
  x: 0.801441490650177
  y: 0.5090281963348389
  z: -0.02190401405096054
}
landmark {
  x: 0.9671028256416321
  y: 0.5659788846969604
  z: -0.0013254738878458738
}
landmark {
  x: 0.8804426193237305
  y: 0.45876678824424744
  z: -0.015764089301228523
}
landmark {
  x: 0.8201411366462708
 

In [12]:
def extract_keypoints(results):
    #for right hand
    rightHand = []
    if results.right_hand_landmarks:
        for res in results.right_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            rightHand.append(test)
        rightHand = np.array(rightHand).flatten()
    else:
        rightHand = np.zeros(21*3)
    print(rightHand.shape)
    
    #for left hand
    leftHand = []
    if results.left_hand_landmarks:
        for res in results.left_hand_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            leftHand.append(test)
        leftHand = np.array(leftHand).flatten()
    else:
        leftHand = np.zeros(21*3)
        
    print(leftHand.shape)
        
    #for face
    face = []
    if results.face_landmarks:
        for res in results.face_landmarks.landmark:
            test = np.array([res.x, res.y, res.z])
            face.append(test)
        face = np.array(face).flatten()
    else:
        face = np.zeros(468*3)
    print(face.shape)
    
    #for pose
    pose = []
    if results.pose_landmarks:
        for res in results.pose_landmarks.landmark:
            test = np.array([res.x, res.y, res.z, res.visibility])
            pose.append(test)
        pose = np.array(pose).flatten()
    else:
        pose = np.zeros(33*4)
    print(pose.shape)
        
    return np.concatenate([pose, face, rightHand, leftHand])

In [13]:
result_test = extract_keypoints(results).shape

(63,)
(63,)
(1404,)
(132,)


In [14]:
np.save('0', result_test)

In [15]:
np.load('0.npy')

array([1662])

# Setup folders for collection

In [16]:
DATA_PATH = os.path.join('MP_DATA')

#actions we are going to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

#30 videos data
no_sequences = 30

#video length = 30
sequence_length = 30

In [17]:
for action in actions:
    for sequence in range(no_sequences):
        try : 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect keypoint values for training and texting

In [18]:
cap = cv2.VideoCapture(0)  #accessing web cam

#setting mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
#     while cap.isOpened(): 
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
                #read from camera
                ret, frame = cap.read()

                #make detections using mediapipe_detection
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                #draw landmarks
                #draw_landmarks(image, results)
                draw_styled_landmarks(image, results)
                
                #wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'COLLECTING frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                    #show to screen
                    cv2.imshow('OpenCV Feed', image)
                else:
                    cv2.putText(image, 'COLLECTING frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    #show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                #export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                

                #if we hit q then our video capture stops
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break        
    cap.release() #release webcam
    cv2.destroyAllWindows() #destroy the window

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(63,)
(63,)
(1404,)
(132,)
(

# PREPROCESS DATA AND CREATE LABELS AND FEATURES

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], [] #sequences is x, labels is y
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.05)

In [ ]:
y_test.shape

# Build and Train LSTM Neural Ntwork

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape= (30,1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [ ]:
model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])  

In [ ]:
model.fit(X_train, y_train, epochs = 2000, callbacks = [tb_callback])

In [ ]:
model.summary()

# Make predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[2])]

In [ ]:
actions[np.argmax(y_test[2])]

# Sve weights

In [ ]:
model.save('action.h5')

# Evaluation using confusion matrix and accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis = 1).tolist()
yhat = np.argmax(yhat, axis = 1).tolist()

In [ ]:
yhat

In [ ]:
ytrue

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
multilabel_confusion_matrix??

In [ ]:
accuracy_score(ytrue, yhat)

# Test in real time

In [ ]:
sequence = []
sentence = []
threshold = 0.4

cap = cv2.VideoCapture(0)  #accessing web cam

#setting mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened(): 
        
        #read from camera
        ret, frame = cap.read()

        #make detections using mediapipe_detection
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)
        
        #preduction logic
        keypoints = extstract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        
        if len(sequence) ==30:
            res = model.predict(np.expand_dims(sequence, axis =0))[0]
            print(actions[np.argmax(res)])
        
        #viulaization logic
        if res[np.argmax(res)] > threshold:
            if len(sentence)>0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
                
            if len(sentence)>5:
                sentence = sentence[-5:]
                
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #show to screen
        cv2.imshow('OpenCV Feed', image)

        #if we hit q then our video capture stops
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break        
    cap.release() #release webcam
    cv2.destroyAllWindows() #destroy the window

In [ ]:
cap.release() #release webcam
cv2.destroyAllWindows()